In [4]:
!git clone https://github.com/princeton-nlp/PURE.git

Cloning into 'PURE'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 61 (delta 16), reused 10 (delta 10), pack-reused 36
Receiving objects: 100% (61/61), 1.59 MiB | 7.41 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [6]:
!mkdir PURE/data
!mkdir PURE/output_ent
!mkdir PURE/output_rel

In [ ]:
!pip install -r PURE/requirements.txt

In [ ]:
# generates ent_pred_dev.json
!python /run_entity.py \
    --do_train --do_eval \
    --learning_rate=1e-5 \
    --task_learning_rate=5e-4 \
    --train_batch_size=16 \
    --context_window 100 \
    --task ace05 \
    --data_dir /content/PURE/data \
    --model bert-base-uncased \
    --output_dir /content/PURE/output_ent

In [ ]:
# generates ent_pred_test.json
!python PURE/run_entity.py \
    --do_train --do_eval --eval_test \
    --learning_rate=1e-5 \
    --task_learning_rate=5e-4 \
    --train_batch_size=16 \
    --context_window 100 \
    --task ace05 \
    --data_dir /content/PURE/data \
    --model bert-base-uncased \
    --output_dir /content/PURE/output_ent

In [1]:
data_folder = "PURE/data2"
output_rel = "PURE/output_rel_ent2"

In [2]:
!python PURE/run_relation.py \
  --task ace05 \
  --do_train --train_file {data_folder}/train.json \
  --do_eval --eval_test \
  --model bert-base-uncased \
  --do_lower_case \
  --train_batch_size 32 \
  --eval_batch_size 32 \
  --learning_rate 2e-5 \
  --num_train_epochs 8 \
  --context_window 300 \
  --max_seq_length 128 \
  --no_cuda \
  --entity_output_dir {data_folder} \
  --output_dir {output_rel}

07/13/2022 15:31:43 - INFO - root - Generate relation data from PURE/data2/train.json
07/13/2022 15:31:43 - INFO - root - #samples: 3214, max #sent.samples: 182
07/13/2022 15:31:43 - INFO - root - Generate relation data from PURE/data2/ent_pred_dev.json
07/13/2022 15:31:43 - INFO - root - #samples: 84, max #sent.samples: 30
07/13/2022 15:31:43 - INFO - root - Generate relation data from PURE/data2/ent_pred_test.json
07/13/2022 15:31:43 - INFO - root - #samples: 112, max #sent.samples: 20
07/13/2022 15:31:43 - INFO - __main__ - ['PURE/run_relation.py', '--task', 'ace05', '--do_train', '--train_file', 'PURE/data2/train.json', '--do_eval', '--eval_test', '--model', 'bert-base-uncased', '--do_lower_case', '--train_batch_size', '32', '--eval_batch_size', '32', '--learning_rate', '2e-5', '--num_train_epochs', '8', '--context_window', '300', '--max_seq_length', '128', '--no_cuda', '--entity_output_dir', 'PURE/data2', '--output_dir', 'PURE/output_rel_ent2']
07/13/2022 15:31:43 - INFO - __main_

In [12]:
#!python PURE/run_eval.py --prediction_file PURE/output_rel/predictions.json
!python PURE/run_eval.py --prediction_file PURE/output_rel_ent2/predictions.json

Evaluation result PURE/output_rel_ent2/predictions.json
NER - P: 1.000000, R: 1.000000, F1: 1.000000
REL - P: 0.702703, R: 0.838710, F1: 0.764706
REL (strict) - P: 0.702703, R: 0.838710, F1: 0.764706


In [11]:
!zip -r output_rel_azure_07_13.zip {output_rel}

  adding: PURE/output_rel_ent2/ (stored 0%)
  adding: PURE/output_rel_ent2/.amlignore (deflated 32%)
  adding: PURE/output_rel_ent2/.amlignore.amltmp (deflated 32%)
  adding: PURE/output_rel_ent2/config.json (deflated 45%)
  adding: PURE/output_rel_ent2/label_list.json (stored 0%)
  adding: PURE/output_rel_ent2/predictions.json (deflated 78%)
  adding: PURE/output_rel_ent2/pytorch_model.bin (deflated 7%)
  adding: PURE/output_rel_ent2/special_tokens.json (deflated 67%)
  adding: PURE/output_rel_ent2/train.log (deflated 90%)
  adding: PURE/output_rel_ent2/vocab.txt (deflated 53%)


In [6]:
import json

file1 = open('PURE/output_rel_ent2/predictions.json', 'r')
lines = file1.readlines()
true_pos = 0
false_pos = 0
false_neg = 0
tot = 0

def calc_true_pos(predicted, actual):
    count = 0
    for p in predicted:
        if p in actual:
            count += 1
    return count

for l in lines:
    d = json.loads(l)
    predicted = d["predicted_relations"][0]
    actual = d["relations"][0]

    #print(" ".join(d["sentences"][0]))
    #print(predicted)
    #print(actual)
    #print("====")
    true_pos += calc_true_pos(predicted, actual)
    #false_pos += calc_false_pos(predicted, actual)
    #false_neg += calc_false_neg(predicted, actual)
    tot += len(actual)

print(true_pos)
print(tot)

26
31
